# Lab 5: Decision Trees

---



## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
from graphviz import Digraph

In [2]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.1 MB/s eta 0:00:00


In [3]:
import category_encoders as ce

## Importing the dataset

In [4]:
data = pd.read_csv("dataset.csv")

In [5]:
data

,Alternate,Bar,Fri/Sat,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,WillWait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,"""0-10""",Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,"""30-60""",No
2,No,Yes,No,No,Some,$,No,No,Burger,"""0-10""",Yes
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,"""10-30""",Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,""">60""",No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,"""0-10""",Yes
6,No,Yes,No,No,None,$,Yes,No,Burger,"""0-10""",No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,"""0-10""",Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,""">60""",No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,"""10-30""",No


## Data preprocessing

In [6]:
# Specify the columns to be encoded
columns_to_encode = ['Alternate', 'Bar', 'Fri/Sat', 'Hungry', 'Patrons', 'Price', 'Raining', 'Reservation', 'Type', 'WaitEstimate']

# Initialize the OrdinalEncoder
encoder = ce.OrdinalEncoder(cols=columns_to_encode)

# Apply the encoding to the specified columns
data_encoded = encoder.fit_transform(data)

data_encoded

,Alternate,Bar,Fri/Sat,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,WillWait
0,1,1,1,1,1,1,1,1,1,1,Yes
1,1,1,1,1,2,2,1,2,2,2,No
2,2,2,1,2,1,2,1,2,3,1,Yes
3,1,1,2,1,2,2,2,2,2,3,Yes
4,1,1,2,2,2,1,1,1,1,4,No
5,2,2,1,1,1,3,2,1,4,1,Yes
6,2,2,1,2,3,2,2,2,3,1,No
7,2,1,1,1,1,3,2,1,2,1,Yes
8,2,2,2,2,2,2,2,2,3,4,No
9,1,2,2,1,2,1,1,1,4,3,No


## Entropy function

In [8]:
def entropy(data):
    _, counts = np.unique(data, return_counts=True) # "_" contains unique elements , "counts" contains corresponding counts
    probabilities = counts / len(data)

    return -np.sum(probabilities * np.log2(probabilities + 1e-10))

### Example

In [9]:
data = np.array([1, 1, 0, 1, 0, 0, 1, 0, 1, 0])
result = entropy(data)
print(result)

0.9999999997114609


## Information gain function

In [10]:
def information_gain(data, attribute, classes):
    total_entropy = entropy(data[classes])  # calculate the dataset entropy

    unique_values = np.unique(data[attribute])
    weighted_entropy = 0

    # calculate weighted entropy
    for value in unique_values:
        subset = data[data[attribute] == value]
        subset_entropy = entropy(subset[classes])
        weight = len(subset) / len(data)
        weighted_entropy += weight * subset_entropy

    return total_entropy - weighted_entropy

## Decision tree class

In [7]:
class DecisionTree:
    def __init__(self, attribute=None, value=None, decision=None):
        self.attribute = attribute
        self.value = value
        self.decision = decision
        self.children = {}

def build_decision_tree(data, attributes, classes):
    # If all examples have the same class, return a leaf node with that class
    if len(np.unique(data[classes])) == 1:
        return DecisionTree(decision=np.unique(data[classes])[0])

    # If no attributes are left, return a leaf node with the majority class
    if len(attributes) == 0:
        majority_class = np.argmax(np.bincount(data[classes]))
        return DecisionTree(decision=majority_class)

    # Choose the best attribute to split on based on information gain
    best_attribute = max(attributes, key=lambda attr: information_gain(data, attr, classes))

    # Create a new decision tree node with the chosen attribute
    tree = DecisionTree(attribute=best_attribute)

    # Recursively build the tree for each possible value of the chosen attribute
    for value in np.unique(data[best_attribute]):
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            # If no examples for a value, create a leaf node with the majority class
            majority_class = np.argmax(np.bincount(data[classes]))
            tree.children[value] = DecisionTree(decision=majority_class)
        else:
            # Recursively build the subtree
            new_attributes = [attr for attr in attributes if attr != best_attribute]
            tree.children[value] = build_decision_tree(subset, new_attributes, classes)

    return tree

## Predict function

In [12]:
def predict(tree, example):
    if tree.decision is not None:
        return tree.decision
    else:
        value = example[tree.attribute]
        if value not in tree.children:
            # If the value is not in the training set, return the majority class
            return tree.children[list(tree.children.keys())[0]].decision
        return predict(tree.children[value], example)

## Validate function

In [13]:
def validate(tree, validation_set, classes):
    correct_predictions = 0
    for _, example in validation_set.iterrows():
        prediction = predict(tree, example)
        if prediction == example[classes]:
            correct_predictions += 1
    return correct_predictions

## Print decision tree function

In [15]:
def print_decision_tree(tree, attribute_map, dot=None):
    if dot is None:
        dot = Digraph(comment='Decision Tree')

    if tree.decision is not None:
        dot.node(str(tree.decision), shape='box')
    else:
        dot.node(tree.attribute)

        for value, child in tree.children.items():
            if child.decision is not None:
                dot.node(str(child.decision), shape='box')
            else:
                dot.node(child.attribute)
                print_decision_tree(child, attribute_map, dot)

            # Map encoded value to real value
            if tree.attribute in attribute_map:
                real_value = attribute_map[tree.attribute][value]
            else:
                real_value = value

            dot.edge(tree.attribute, str(child.decision) if child.decision is not None else child.attribute, label=str(real_value))

    return dot


## Applying Decision Tree on given dataset

In [16]:
# Split the dataset into training and validation sets
training_set = data_encoded.sample(n=10, random_state=42)
validation_set = data_encoded.drop(training_set.index)

In [17]:
training_set

,Alternate,Bar,Fri/Sat,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,WillWait
10,2,1,1,2,3,2,1,2,2,1,No
9,1,2,2,1,2,1,1,1,4,3,No
0,1,1,1,1,1,1,1,1,1,1,Yes
8,2,2,2,2,2,2,2,2,3,4,No
5,2,2,1,1,1,3,2,1,4,1,Yes
2,2,2,1,2,1,2,1,2,3,1,Yes
1,1,1,1,1,2,2,1,2,2,2,No
11,1,2,2,1,2,2,1,2,3,2,Yes
4,1,1,2,2,2,1,1,1,1,4,No
7,2,1,1,1,1,3,2,1,2,1,Yes


In [18]:
validation_set

,Alternate,Bar,Fri/Sat,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,WillWait
3,1,1,2,1,2,2,2,2,2,3,Yes
6,2,2,1,2,3,2,2,2,3,1,No


In [20]:
encoded_attributes = list(data_encoded.columns.difference(['WillWait']))
encoded_classes = 'WillWait'

# Build the decision tree
tree = build_decision_tree(training_set, encoded_attributes, encoded_classes)

# Print the decision tree using Graphviz
# dot = print_decision_tree(tree)
# dot.render('decision_tree', format='png', cleanup=True)

# Define mappings from encoded values to real values for each attribute
attribute_map = {
    'Alternate': {1: 'Yes', 2: 'No'},
    'Bar': {1: 'Yes', 2: 'No'},
    'Fri/Sat': {1: 'Yes', 2: 'No'},
    'Hungry': {1: 'Yes', 2: 'No'},
    'Patrons': {1: 'Some', 2: 'Full', 3: 'None'},
    'Price': {1: '$', 2: '$$', 3: '$$$'},
    'Raining': {1: 'Yes', 2: 'No'},
    'Reservation': {1: 'Yes', 2: 'No'},
    'Type': {1: 'French', 2: 'Thai', 3: 'Burger', 4: 'Italian'},
    'WaitEstimate': {1: '0-10', 2: '10-30', 3: '30-60', 4: '>60'},
}

# Print the decision tree using real values
dot = print_decision_tree(tree, attribute_map)
dot.render('decision_tree_real_values', format='png', cleanup=True)

'decision_tree_real_values.png'

### Output

Make sure to open this notebook in Google Colab.

![picture](https://drive.google.com/uc?export=view&id=1H7VnZ2G6Uvbfp4zWrEU50xKDzQcUbSRm)

### Validating on the validation set

In [21]:
accuracy = validate(tree, validation_set, encoded_classes)
print(f'Accuracy on validation set: {accuracy}/{len(validation_set)}')

Accuracy on validation set: 1/2
